In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.environ["GEMINI_API_KEY"],
)

# Intro

In [ ]:
basic_prompt = "Explain the concept of prompt engineering in one sentence."
print(llm.invoke(basic_prompt).content)

In [ ]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Provide a definition of {topic}, explain its importance, and list three key benefits."
)

chain = structured_prompt | llm # Combine the prompt template with the language model
input_variables = {"topic": "prompt engineering"} # Define the input variables
output = chain.invoke(input_variables).content # Invoke the chain with the input variables
print(output)

# Multi-turn Prompts

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Chain the prompt and the LLM
chain = prompt | llm

# Create a runnable with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: ChatMessageHistory(),
    input_messages_key="input",
    history_messages_key="history",
)

# Example usage
conversation_id = "space_conversation"
messages = []

# First query
response1 = chain_with_history.invoke(
    {"input": "Hi, I'm learning about space. Can you tell me about planets?"}, 
    config={"configurable": {"session_id": conversation_id}}
)
print(response1.content)

# Second query
response2 = chain_with_history.invoke(
    {"input": "What's the largest planet in our solar system?"}, 
    config={"configurable": {"session_id": conversation_id}}
)
print(response2.content)

# Third query
response3 = chain_with_history.invoke(
    {"input": "How does its size compare to Earth?"}, 
    config={"configurable": {"session_id": conversation_id}}
)
print(response3.content)

# Templating

In [ ]:
from jinja2 import Template
from langchain_core.messages import HumanMessage

In [ ]:
def get_completion(prompt):
    ''' Get a completion from the Gemini API using Langchain
    Args:
        prompt (str): The prompt to send to the API

    Returns:
        str: The completion text
    '''
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    return response.content

In [ ]:
class PromptTemplate:
    ''' A class to represent a template for generating prompts with variables
    Attributes:
        template (str): The template string with variables
        input_variables (list): A list of the variable names in the template
    '''
    def __init__(self, template, input_variables):
        self.template = Template(template)
        self.input_variables = input_variables
    
    def format(self, **kwargs):
        return self.template.render(**kwargs)

In [ ]:
simple_template = PromptTemplate(
    template="Provide a brief explanation of {{ topic }}.",
    input_variables=["topic"]
)

In [ ]:
print("Simple Template Result:")
prompt = simple_template.format(topic="photosynthesis")
print(get_completion(prompt))

In [ ]:
complex_template = PromptTemplate(
    template="Explain the concept of {{ concept }} in the field of {{ field }} to a {{ audience }} audience, concisely.",
    input_variables=["concept", "field", "audience"]
)

In [ ]:
print("Complex Template Result:")
prompt = complex_template.format(
    concept="neural networks",
    field="artificial intelligence",
    audience="beginner"
)
print(get_completion(prompt))

In [ ]:
conditional_template = PromptTemplate(
    template="My name is {{ name }} and I am {{ age }} years old. "
              "{% if profession %}I work as a {{ profession }}.{% else %}I am currently not employed.{% endif %} "
              "Can you give me career advice based on this information? answer concisely.",
    input_variables=["name", "age", "profession"]
)

# Using the conditional template
print("Conditional Template Result (with profession):")
prompt = conditional_template.format(
    name="Alex",
    age="28",
    profession="software developer"
)
print(get_completion(prompt))

In [ ]:
print("\nConditional Template Result (without profession):")
prompt = conditional_template.format(
    name="Sam",
    age="22",
    profession=""
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

In [ ]:
list_format_template = PromptTemplate(
    template="Analyze the following list of items:\n"
              "{% for item in items.split(',') %}"
              "- {{ item.strip() }}\n"
              "{% endfor %}"
              "\nProvide a summary of the list and suggest any patterns or groupings.",
    input_variables=["items"]
)


# Using the formatted list template
print("Formatted List Template Result:")
prompt = list_format_template.format(
    items="Python, JavaScript, HTML, CSS, React, Django, Flask, Node.js"
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")